In [1]:
import rasterio as rio
import rioxarray as rxr

import pystac_client
import pystac
import planetary_computer
from odc.geo.geom import BoundingBox
import odc.geo.xr
import odc.stac
from odc.stac import configure_rio
from datacube.utils.aws import configure_s3_access
from distributed import LocalCluster, Client

import geopandas as gpd

from pystac_client.stac_api_io import StacApiIO
from urllib3 import Retry

In [2]:
cluster = LocalCluster(
    n_workers=16, 
    threads_per_worker=1, 
    processes=False,
    # memory_limit='4GB',
    # local_directory="/tmp/dask-worker-space",
    )
client = Client(cluster)
configure_rio(cloud_defaults=True, client=client) # For Planetary Computer
print(f'The Dask client listens to {client.dashboard_link}')

c:\Users\fotakidis\anaconda3\envs\drought\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59304 instead
  warnings.warn(


The Dask client listens to http://155.207.39.39:59304/status


2025-08-14 23:31:59,207 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.97 GiB -- Worker memory limit: 3.99 GiB


In [3]:
aoi = gpd.read_file('../anciliary/grid_v2.geojson').to_crs('EPSG:4326')
# Get the bounds of the 9th geometry
minx, miny, maxx, maxy = aoi.iloc[7].geometry.bounds
print(aoi.iloc[7].tile_ids)
aoi_bbox = BoundingBox.from_xy(
    (minx, maxx),
    (miny, maxy)
)

x06_y10


In [16]:
stac_api_io = StacApiIO(max_retries=Retry(total=5, backoff_factor=5))

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    stac_io=stac_api_io
)

cloud_cover = 70
search = catalog.search(
    collections=["sentinel-2-l2a"], #hls2-s30
    bbox=aoi_bbox,
    datetime='2025-07-15/2025-08-14',
    limit=100,
    query={
        "eo:cloud_cover": {"lt":cloud_cover},
        "s2:nodata_pixel_percentage": {"lt":20},
    },
)

items = search.item_collection()

In [17]:
BANDS = ['B03','B04']
EPSG = 'EPSG:32634'
RESOLUTION = 10
RESAMPLING_ALGO = "average"

ds_cube = odc.stac.stac_load(
    items,
    bbox=aoi_bbox,
    bands=BANDS,
    chunks=dict(y=2048, x=2048),
    crs=EPSG,  # {epsgs[0]}
    resolution=RESOLUTION,
    groupby='time', # if 'time' loads all items, retaining duplicates
    fail_on_error=True,
    resampling={
        "*": RESAMPLING_ALGO,
    },
).compute()


ds_cube = ds_cube.astype('uint16')
ds_cube['B04'] = ds_cube['B04'].rio.write_nodata(0, inplace=True)

ds_cube['B03'] = ds_cube['B03'].rio.write_nodata(0, inplace=True)

# ds_cube['B04'].rio.to_raster('b04-10m-none-lossless.tif',
#                              driver='COG',
#                              )

c:\Users\fotakidis\anaconda3\envs\drought\Lib\site-packages\xarray\core\duck_array_ops.py:228: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [18]:
import numpy as np

def s2_downsample_dataset_10m_to_20m(ds: xr.Dataset, y="y", x="x") -> xr.Dataset:
    """
    Sen2Cor-style 10m -> 20m downsampling for all variables in a Dataset:
      - 2x2 block mean over (y, x)
      - round-half-up (+0.5) for integer vars
      - cast back to original dtype for those integer vars
    Time (and any other) dims are preserved.
    """
    if y not in ds.dims or x not in ds.dims:
        raise ValueError(f"Dataset must have '{y}' and '{x}' dims.")

    # Coarsen spatially; boundary='trim' drops the last row/col if odd-sized.
    coarsened = ds.coarsen({y: 2, x: 2}, boundary="trim").mean()

    # Round/cast back for integer-typed variables (e.g., uint16 Sentinel-2 DNs)
    out = coarsened.copy()
    for name, da in coarsened.data_vars.items():
        orig_dt = ds[name].dtype
        if np.issubdtype(orig_dt, np.integer):
            out[name] = (da + 0.5).astype(orig_dt)

    return out

In [19]:
ds20 = s2_downsample_dataset_10m_to_20m(ds_cube, y="y", x="x")

In [23]:
ds20.isel(time=-1).B04.rio.to_raster('B04_block_reduce_from ds.tif')

In [62]:
import rasterio, numpy as np
from skimage.measure import block_reduce
from numpy import mean, uint16


with rasterio.open("b04-10m-none-lossless.tif") as src:
    arr = src.read(1).astype(float)
    arrb = uint16(block_reduce(arr, block_size=(2, 2), func=mean) + 0.5)
    
    
    
    
    # nodata = src.nodata if src.nodata is not None else 0
    # arr[arr == nodata] = np.nan
    # out = block_reduce(arr, block_size=(2,2), func=np.nanmedian)
    # profile = src.profile
    # profile.update(height=out.shape[0], width=out.shape[1], transform=src.transform * src.transform.scale(2,2))
    # with rasterio.open("B04_20m_from10m_med.tif", "w", **profile) as dst:
    #     out = np.nan_to_num(out, nan=nodata).astype(src.dtypes[0])
    #     dst.write(out, 1)


In [8]:
from skimage.measure import block_reduce
from numpy import mean, uint16
import xarray as xr

def s2_downsample_10m_to_20m(da: xr.DataArray, y="y", x="x") -> xr.DataArray:
    # Expect uint16 DNs; zeros are valid samples (no special NoData treatment).
    if da.sizes[y] < 2 or da.sizes[x] < 2:
        raise ValueError("Array must be at least 2x2 in the spatial dims.")

    # Trim to an even size so 2x2 blocks line up exactly
    H2 = da.sizes[y] // 2
    W2 = da.sizes[x] // 2
    da = da.isel({y: slice(0, H2 * 2), x: slice(0, W2 * 2)})

    # Mean over 2x2 windows, then round-half-up and cast to uint16
    out = da.coarsen({y: 2, x: 2}, boundary="trim").mean()      # float
    out = (out + 0.5).astype(uint16)                          # round-half-up

    return out

In [9]:
out = s2_downsample_10m_to_20m(ds_cube.B04)

In [11]:
ds_cube.B04

<xarray.DataArray 'B04' (time: 1, y: 5471, x: 5352)> Size: 59MB
array([[[1360, 1381, 1332, ..., 1538, 1578, 1552],
        [1425, 1400, 1368, ..., 1556, 1528, 1544],
        [1374, 1372, 1385, ..., 1547, 1518, 1514],
        ...,
        [1602, 1522, 1620, ..., 1715, 1658, 1646],
        [1583, 1540, 1607, ..., 1702, 1667, 1680],
        [1532, 1612, 1588, ..., 1670, 1665, 1676]]], dtype=uint16)
Coordinates:
  * y            (y) float64 44kB 4.262e+06 4.262e+06 ... 4.208e+06 4.208e+06
  * x            (x) float64 43kB 4.424e+05 4.425e+05 ... 4.959e+05 4.96e+05
    spatial_ref  int32 4B 32634
  * time         (time) datetime64[ns] 8B 2025-08-13T09:30:39.024000
Attributes:
    _FillValue:  0

In [12]:
out.rio.to_raster('B04_block_reduce.tif')

In [13]:
import numpy as np
import xarray as xr
from skimage.measure import block_reduce

def s2_downsample_with_block_reduce(da: xr.DataArray, y="y", x="x") -> xr.DataArray:
    H2 = da.sizes[y] // 2
    W2 = da.sizes[x] // 2
    da = da.isel({y: slice(0, H2 * 2), x: slice(0, W2 * 2)})

    def f(a):
        return np.uint16(block_reduce(a, block_size=(2, 2), func=np.mean) + 0.5)

    out = xr.apply_ufunc(
        f, da,
        input_core_dims=[[y, x]],
        output_core_dims=[[y, x]],
        output_sizes={y: H2, x: W2},
        dask="parallelized",
        output_dtypes=[np.uint16],
        vectorize=False,
    )
    return out

In [14]:
out = s2_downsample_10m_to_20m(ds_cube.B04)

In [15]:
out.rio.to_raster('B04_block_reduce.tif')

In [58]:
BANDS = ['B04']
EPSG = 'EPSG:32634'
RESOLUTION = 20
RESAMPLING_ALGO = "cubic"

ds_cube = odc.stac.stac_load(
    items,
    bbox=aoi_bbox,
    bands=BANDS,
    chunks=dict(y=2048, x=2048),
    crs=EPSG,  # {epsgs[0]}
    resolution=RESOLUTION,
    groupby='time', # if 'time' loads all items, retaining duplicates
    fail_on_error=True,
    resampling={
        "*": RESAMPLING_ALGO,
    },
).compute()


ds_cube = ds_cube.astype('uint16')
ds_cube['B04'] = ds_cube['B04'].rio.write_nodata(0, inplace=True)

ds_cube['B04'].rio.to_raster(f'b04-{RESOLUTION}m-{RESAMPLING_ALGO}.tif',
                             driver='COG',
                             )

In [ ]:
BANDS = ['B05']
EPSG = 'EPSG:32634'
RESOLUTION = 20
RESAMPLING_ALGO = "average"

ds_cube = odc.stac.stac_load(
    items,
    bbox=aoi_bbox,
    bands=BANDS,
    chunks=dict(y=2048, x=2048),
    crs=EPSG,  # {epsgs[0]}
    resolution=RESOLUTION,
    groupby='time', # if 'time' loads all items, retaining duplicates
    fail_on_error=True,
    resampling={
        "*": RESAMPLING_ALGO,
    },
).compute()
ds_cube = ds_cube.astype('uint16')
ds_cube['B05'] = ds_cube['B05'].rio.write_nodata(0, inplace=True)
ds_cube['B05'].rio.to_raster('b05-20m-none-lossless.tif',
                             driver='COG',
                             )

DriverRegistrationError: ('No such driver registered: %s', b'JP2OpenJPEG')